In [ ]:
import os  ### This imports the python's built in module called os. Work with files and folders (example: reading secret keys stored safely)
from openai import OpenAI ### This imports the OpenAI client class from the openai library.this is what lets your code talk to OpenAI.
from dotenv import load_dotenv ###This imports a function that can read a .env file.
load_dotenv(override=True) ### load_dotenv(override=True) loads your secret keys from the .env file and makes sure your program uses those values.

True

In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"   ### “This is the address where Gemini AI lives on the internet.”Think of it like writing the home address of Gemini so we know where to send questions.
google_api_key = os.getenv("GOOGLE_API_KEY")
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=[{"role":"user", "content": "what is 2+2?"}])
print(response.choices[0].message.content)

2 + 2 = 4


In [ ]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = """
You are a snarkyassistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

In [6]:
# Define our user prompt

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

response = gemini.chat.completions.create(model="gemini-2.5-flash", messages=messages)
response.choices[0].message.content

'2 + 2 = 4'

In [13]:
from bs4 import BeautifulSoup
import requests


# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]


In [14]:
ed = fetch_website_contents("https://edwarddonner.com")
print(ed)

Home - Edward Donner

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Conne

In [15]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [16]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarkyassistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nHome - Edward Donner\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. 

In [21]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = fetch_website_contents(url)
    response = gemini.chat.completions.create(
        model = "gemini-2.5-flash",
        messages = messages_for(website)
    )
    return response.choices[0].message.content  

In [22]:
summarize("https://edwarddonner.com")

'Welcome to Edward Donner\'s digital abode, where he\'s just your average AI-obsessed CTO who built a company that got acquired and is now building another. When not pondering the mysteries of LLMs (and setting them up for a battle royale of diplomacy in "Outsmart"), he\'s busy DJing badly, making "very amateur" electronic music, and nodding sagely to things he doesn\'t quite understand on Hacker News.\n\n**Future Forecasts (aka News from a Time Traveler):**\nApparently, Ed has access to a TARDIS, because he\'s already posted about "The Unique Energy of an AI Live Event" (Nov 11, 2025), "AI in Production: Gen AI and Agentic AI on AWS at scale" (Sep 15, 2025), a curriculum to "Be an AI Engineer and Leader" (May 28, 2025), and a "2025 AI Executive Briefing" (May 18, 2025). So, if you need to know what\'s happening in AI *next year*, he\'s your go-to.'

In [27]:
# A function to display this nicely in the output, using markdown
from IPython.display import Markdown, display
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [28]:
display_summary("https://edwarddonner.com")

Edward Donner is an AI big shot who casually drops that he's a CTO, acquired a startup, and holds patents, but also confesses to being a "very amateur" musician and a frequent *Hacker News* reader who pretends to understand things. He's also built a digital Thunderdome where LLMs can duke it out.

**Announcements (or prophecies from the future):**
Apparently, Ed has a crystal ball (or just excellent planning skills), because he's already lined up a series of AI thought leadership posts for 2025, including insights on AI live events, production at scale on AWS, an AI engineer curriculum, and an executive briefing. Get ready for tomorrow's news, today!